# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,8.23,81,100,5.16,NZ,1693058857
1,1,kursavka,44.4564,42.5093,26.89,27,10,4.99,RU,1693058858
2,2,edinburgh of the seven seas,-37.0676,-12.3116,15.21,85,99,15.22,SH,1693058859
3,3,ancud,-41.8697,-73.8203,5.26,72,6,2.47,CL,1693058859
4,4,blackmans bay,-43.0167,147.3167,4.29,88,50,1.68,AU,1693058665


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_city_data= city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=600, frame_height=500, size="Humidity", scale=0.8, color="City")

# Display the map
map_city_data

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_temp_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21)]
filtered_windspeed_df = filtered_temp_df.loc[filtered_temp_df["Wind Speed"]<4.5]
filtered_Cloudiness_df = filtered_windspeed_df.loc[filtered_windspeed_df["Cloudiness"]==0]

# Drop any rows with null values
filtered_df = filtered_Cloudiness_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,pahrump,36.2083,-115.9839,21.03,29,0,1.66,US,1693058913
81,81,colonia,50.9333,6.9500,24.47,47,0,3.09,DE,1693058850
84,84,kununurra,-15.7667,128.7333,24.99,22,0,3.60,AU,1693058926
159,159,carnarvon,-24.8667,113.6333,21.46,56,0,3.03,AU,1693058983
210,210,toliara,-23.3500,43.6667,25.43,65,0,3.17,MG,1693059047
372,372,katsuura,35.1333,140.3000,25.81,85,0,1.91,JP,1693059171


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#newdf = filtered_df.copy()
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.reset_index(drop=True)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pahrump,US,36.2083,-115.9839,29,
1,colonia,DE,50.9333,6.9500,47,
2,kununurra,AU,-15.7667,128.7333,22,
3,carnarvon,AU,-24.8667,113.6333,56,
4,toliara,MG,-23.3500,43.6667,65,
5,katsuura,JP,35.1333,140.3000,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel" 
limit = 20

params = {"categories":categories,
          "apiKey":geoapify_key,
          "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pahrump - nearest hotel: Holiday Inn Express & Suites
colonia - nearest hotel: Wasserturm Hotel Cologne
kununurra - nearest hotel: Kununurra Country Club Resort
carnarvon - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
katsuura - nearest hotel: 潮騒の宿


,City,Country,Lat,Lng,Humidity,Hotel Name
64,pahrump,US,36.2083,-115.9839,29,Holiday Inn Express & Suites
81,colonia,DE,50.9333,6.9500,47,Wasserturm Hotel Cologne
84,kununurra,AU,-15.7667,128.7333,22,Kununurra Country Club Resort
159,carnarvon,AU,-24.8667,113.6333,56,No hotel found
210,toliara,MG,-23.3500,43.6667,65,Ambary
372,katsuura,JP,35.1333,140.3000,85,潮騒の宿


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)